# Adding columns

In [85]:
import pandas as pd

In [86]:
file_paths = 'cleaned_data.csv', 'cleaned_data_without_outliers.csv'
data_with_outliers, data_without_outliers = pd.read_csv(file_paths[0]), pd.read_csv(file_paths[1])

In [87]:
# Add a new column for price per square meter, avoiding division by zero or null landsize values
data_with_outliers['Land_price_per_m2'] = data_with_outliers.apply(
    lambda row: row['Price'] / row['Landsize'] if pd.notnull(row['Price']) and pd.notnull(row['Landsize']) and row['Landsize'] > 0 else None,
    axis=1
)

data_without_outliers['Land_price_per_m2'] = data_without_outliers.apply(
    lambda row: row['Price'] / row['Landsize'] if pd.notnull(row['Price']) and pd.notnull(row['Landsize']) and row['Landsize'] > 0 else None,
    axis=1
)

# Display the updated dataset's first few rows to confirm the addition
data_with_outliers[['Price', 'Landsize', 'Land_price_per_m2']].head()

,Price,Landsize,Land_price_per_m2
0,1480000.0,202.0,7326.732673
1,1035000.0,156.0,6634.615385
2,1465000.0,134.0,10932.835821
3,850000.0,94.0,9042.553191
4,1600000.0,120.0,13333.333333


In [88]:
# Add a new column for price per square meter, avoiding division by zero or null landsize values
data_with_outliers['Building_price_per_m2'] = data_with_outliers.apply(
    lambda row: row['Price'] / row['BuildingArea'] if pd.notnull(row['Price']) and pd.notnull(row['BuildingArea']) and row['BuildingArea'] > 0 else None,
    axis=1
)

data_without_outliers['Building_price_per_m2'] = data_without_outliers.apply(
    lambda row: row['Price'] / row['BuildingArea'] if pd.notnull(row['Price']) and pd.notnull(row['BuildingArea']) and row['BuildingArea'] > 0 else None,
    axis=1
)

# Display the updated dataset's first few rows to confirm the addition
data_with_outliers[['Price', 'BuildingArea', 'Building_price_per_m2']].head()

,Price,BuildingArea,Building_price_per_m2
0,1480000.0,154.6,9573.091850
1,1035000.0,79.0,13101.265823
2,1465000.0,150.0,9766.666667
3,850000.0,117.4,7240.204429
4,1600000.0,142.0,11267.605634


In [89]:
# Add a new column for price per square meter, avoiding division by zero or null landsize values
data_with_outliers['Building_and_land_price_per_m2'] = data_with_outliers.apply(
    lambda row: row['Price'] / (row['BuildingArea'] + row['Landsize']) if pd.notnull(row['Price']) and pd.notnull(row['Landsize']) and row['Landsize'] > 0 and pd.notnull(row['BuildingArea']) and row['BuildingArea'] > 0 else None,
    axis=1
)

data_without_outliers['Building_and_land_price_per_m2'] = data_without_outliers.apply(
    lambda row: row['Price'] / (row['BuildingArea'] + row['Landsize']) if pd.notnull(row['Price']) and pd.notnull(row['Landsize']) and row['Landsize'] > 0 and pd.notnull(row['BuildingArea']) and row['BuildingArea'] > 0 else None,
    axis=1
)

# Display the updated dataset's first few rows to confirm the addition
data_with_outliers[['Price', 'BuildingArea', 'Landsize', 'Building_and_land_price_per_m2']].head()

,Price,BuildingArea,Landsize,Building_and_land_price_per_m2
0,1480000.0,154.6,202.0,4150.308469
1,1035000.0,79.0,156.0,4404.255319
2,1465000.0,150.0,134.0,5158.450704
3,850000.0,117.4,94.0,4020.813623
4,1600000.0,142.0,120.0,6106.870229


Check correlation of building price per mq2 and land price per mq2 (only calculated on data w\out outliers)

In [90]:
# Filter the data to include only rows where Building_price_per_m2 is not NaN
filtered_data = data_with_outliers.dropna(subset=['Building_price_per_m2'])

# Calculate the correlation between Building_price_per_m2 and Land_price_per_m2
correlation = filtered_data[['Building_price_per_m2', 'Land_price_per_m2']].corr()

# Display the correlation matrix
print(correlation)

# Calculate the correlation between Building_price_per_m2, Land_price_per_m2, and Building_and_land_price_per_m2
correlation = filtered_data[['Building_price_per_m2', 'Land_price_per_m2', 'Building_and_land_price_per_m2']].corr()

# Display the correlation matrix
print('\n\n', correlation)

                       Building_price_per_m2  Land_price_per_m2
Building_price_per_m2               1.000000           0.228343
Land_price_per_m2                   0.228343           1.000000


                                 Building_price_per_m2  Land_price_per_m2  \
Building_price_per_m2                        1.000000           0.228343   
Land_price_per_m2                            0.228343           1.000000   
Building_and_land_price_per_m2               0.292848           0.827100   

                                Building_and_land_price_per_m2  
Building_price_per_m2                                 0.292848  
Land_price_per_m2                                     0.827100  
Building_and_land_price_per_m2                        1.000000  


In [91]:
# Convert the 'Date' column to datetime format
data_with_outliers['Date'] = pd.to_datetime(data_with_outliers['Date'], format='%d/%m/%Y')
data_without_outliers['Date'] = pd.to_datetime(data_without_outliers['Date'], format='%d/%m/%Y')

data_with_outliers['Date'].head()

0   2016-12-03
1   2016-02-04
2   2017-03-04
3   2017-03-04
4   2016-06-04
Name: Date, dtype: datetime64[ns]

to check if the yearbuilt floating numbers, aside from the nan records, are all actual integers (so they're floating with .0 decimal):

In [92]:
all_integers = data_with_outliers['YearBuilt'].dropna().apply(lambda x: x.is_integer()).all()

print(f"All 'YearBuilt' values are integers with .0 floating point: {all_integers}")


All 'YearBuilt' values are integers with .0 floating point: False


In [93]:
data_with_outliers['YearBuilt'] = data_with_outliers['YearBuilt'].round().astype('Int64')
data_without_outliers['YearBuilt'] = data_without_outliers['YearBuilt'].round().astype('Int64')

data_with_outliers['YearBuilt'].head()

0    1896
1    1900
2    1900
3    1962
4    2014
Name: YearBuilt, dtype: Int64

to check that also for property count, we are not converting values that might have a decimal part:

In [94]:
all_integers_property_count = data_with_outliers['Propertycount'].apply(lambda x: x.is_integer()).all()

print(f"All 'Propertycount' values are integers with .0 floating point: {all_integers_property_count}")

All 'Propertycount' values are integers with .0 floating point: True


In [95]:
data_with_outliers['Propertycount'] = data_with_outliers['Propertycount'].astype('Int64')
data_without_outliers['Propertycount'] = data_without_outliers['Propertycount'].astype('Int64')

data_with_outliers['Propertycount'].head()


0    4019
1    4019
2    4019
3    4019
4    4019
Name: Propertycount, dtype: Int64

(i still have to understand wtf is propertycount...)

next: the dummy vars for the regions :)

In [96]:
# Create dummy variables for the 'Regionname' column
region_dummies_for_outliers = pd.get_dummies(data_with_outliers['Regionname'], drop_first=True)
region_dummies_for_no_outliers = pd.get_dummies(data_without_outliers['Regionname'], drop_first=True)

# Display the first few rows of the dummy variables
region_dummies_for_outliers.head()

,Eastern Victoria,Northern Metropolitan,Northern Victoria,South-Eastern Metropolitan,Southern Metropolitan,Western Metropolitan,Western Victoria
0,False,True,False,False,False,False,False
1,False,True,False,False,False,False,False
2,False,True,False,False,False,False,False
3,False,True,False,False,False,False,False
4,False,True,False,False,False,False,False


In [97]:
region_dummies_for_outliers = region_dummies_for_outliers.rename(columns={
    'Eastern Victoria': 'is_Eastern_Victoria',
    'Northern Metropolitan': 'is_Northern_Metropolitan',
    'Northern Victoria': 'is_Northern_Victoria',
    'South-Eastern Metropolitan': 'is_South_Eastern_Metropolitan',
    'Southern Metropolitan': 'is_Southern_Metropolitan',
    'Western Metropolitan': 'is_Western_Metropolitan',
    'Western Victoria': 'is_Western_Victoria'
})

region_dummies_for_no_outliers = region_dummies_for_no_outliers.rename(columns={
    'Eastern Victoria': 'is_Eastern_Victoria',
    'Northern Metropolitan': 'is_Northern_Metropolitan',
    'Northern Victoria': 'is_Northern_Victoria',
    'South-Eastern Metropolitan': 'is_South_Eastern_Metropolitan',
    'Southern Metropolitan': 'is_Southern_Metropolitan',
    'Western Metropolitan': 'is_Western_Metropolitan',
    'Western Victoria': 'is_Western_Victoria'
})

# Display the first few rows of the renamed dummy variables
region_dummies_for_outliers.head()

,is_Eastern_Victoria,is_Northern_Metropolitan,is_Northern_Victoria,is_South_Eastern_Metropolitan,is_Southern_Metropolitan,is_Western_Metropolitan,is_Western_Victoria
0,False,True,False,False,False,False,False
1,False,True,False,False,False,False,False
2,False,True,False,False,False,False,False
3,False,True,False,False,False,False,False
4,False,True,False,False,False,False,False


In [98]:
# Concatenate the region dummies with the original data dataframe
data_with_outliers = pd.concat([data_with_outliers, region_dummies_for_outliers], axis=1)
data_without_outliers = pd.concat([data_without_outliers, region_dummies_for_no_outliers], axis=1)

# Display the dataframe after the addition
print('Data info, dataset with outliers:')
print(data_with_outliers.info())

print('\n\nData info, dataset without outliers:')
print(data_without_outliers.info())

Data info, dataset with outliers:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13518 entries, 0 to 13517
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Suburb                          13518 non-null  object        
 1   Address                         13518 non-null  object        
 2   Rooms                           13518 non-null  int64         
 3   Type                            13518 non-null  object        
 4   Price                           13518 non-null  float64       
 5   Method                          13518 non-null  object        
 6   SellerG                         13518 non-null  object        
 7   Date                            13518 non-null  datetime64[ns]
 8   Distance                        13518 non-null  float64       
 9   Postcode                        13518 non-null  float64       
 10  Bedroom2                        1351

In [99]:
# Extract the year from the 'Date' column
data_with_outliers['YearSold'] = data_with_outliers['Date'].dt.year
data_without_outliers['YearSold'] = data_without_outliers['Date'].dt.year

# Calculate the age of the house
data_with_outliers['HouseAge'] = data_with_outliers['YearSold'] - data_with_outliers['YearBuilt']
data_without_outliers['HouseAge'] = data_without_outliers['YearSold'] - data_without_outliers['YearBuilt']

# Display the updated dataset's first few rows to confirm the addition
data_with_outliers[['YearBuilt', 'YearSold', 'HouseAge']].head()

,YearBuilt,YearSold,HouseAge
0,1896,2016,120
1,1900,2016,116
2,1900,2017,117
3,1962,2017,55
4,2014,2016,2


In [100]:
data_with_outliers.to_csv('cleaned_and_complete_data.csv', index=False)

In [101]:
data_without_outliers.to_csv('cleaned_and_complete_data_without_outliers.csv', index=False)